# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 0: Preparation and import data from s3
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
#Before we continue, we need to install related python package.
import sys

!{sys.executable} -m pip install boto3
!{sys.executable} -m pip install s3fs
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install cqlsh

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# Do all imports and installs here
import configparser
import pandas as pd
import os
import boto3
import uuid
from time import sleep

In [3]:
config = configparser.ConfigParser()
config.read('iam.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['AWS_CREDS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS_CREDS']['AWS_SECRET_ACCESS_KEY']

client=boto3.client('s3')


### Scope the Project and Gather Data

#### Project description:

This project will be separate to multiple parts, and all four dataset will be used. 

Before we talk about the details, we need to know the characteristics of relational DB and non-relational DB.

For relational DB, its characteristics is low redundancy and high completeness, which means it is very suitable for small or medium size data, and the database does not change so much. In our case, we should store temperature, airport code and US cities demographic data into a relational database that meets 3NF because it does not always change so much and the volume of data is not that large.

For non-relational DB, its characteristics is higher elasticity, faster read & write speed and evoving data volume. In our case, we should save I94 data into non-relational DB. Because this piece of data need to make ETL process almost every minutes in real world background, and it need dynamic write and read for real-time data monitoring.


The final solution will work as a a database management system. When user input the time or time period and the column they interested in (e.g., visa type), the system will return related data as a dataframe. For example, a user needs to know where are the busiest airport for investment visa holder(E-1 visa) in 2016 and its basic information such as temperature, and the status of the city such as population(age, majority race, etc.), or when is the peak-time for international student come to the United States and where are they come from.

* Data will be imported from Amazon S3
* Relational DB will be implement on AWS Redshift
* Non-Relationalship DB will be implement on Amazon Keyspace, and data backup will be stored at S3 as parquet format.
* Data cleaning and ETL process will be implement on Amazon EMR with Spark

#### The dataset is going to use in this project are:

* I94 Immigration Data: This data comes from the US National Tourism and Trade Office. A data dictionary is included in the workspace. https://travel.trade.gov/research/reports/i94/historical/2016.html is where the data comes from. There's a sample file so you can take a look at the data in csv format before reading it all in. You do not have to use the entire dataset, just use what you need to accomplish the goal you set at the beginning of the project.
* World Temperature Data: This dataset came from Kaggle. You can read more about it here: https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data.
* U.S. City Demographic Data: This data comes from OpenSoft. You can read more about it here: https://public.opendatasoft.com/explore/dataset/us-cities-demographics/export/.
* Airport Code Table: This is a simple table of airport codes and corresponding cities. It comes from here:https://datahub.io/core/airport-codes#data.

In [4]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

while True:
    if os.path.isdir('16_jan.parquet'):
        break
    else:
        sql = '''SELECT id_, cicid, i94yr, i94mon, i94cit, i94res, i94port,
                 arrdate, i94mode, i94addr, depdate, i94bir, i94visa,
                 count, dtadfile, visapost, occup, entdepa, entdepd,
                 entdepu, matflag, biryear, dtaddto, gender, insnum,
                 airline, admnum, fltno, visatype FROM i94'''
        
        i94_url='s3://srk-data-eng-capstone/i94/i94_jan16_sub.sas7bdat'
        i94 = pd.read_sas(i94_url, 'sas7bdat',encoding="ISO-8859-1").drop_duplicates()
        i94['id_'] = pd.Series([uuid.uuid1() for each in range(len(i94))])
        i94['dtadfile'].fillna(-1,inplace=True)
        for each in i94:
            dt=i94[each].dtype
            if dt == 'object':
                i94[each].fillna('Null',inplace=True)
            else:
                i94[each].fillna(-1,inplace=True)
        i94.to_csv('16_jan.csv', index=False)
        df_spark=spark.read.option('header','true').csv('16_jan.csv')
        df_spark.createOrReplaceTempView('i94')
        parquet_data=spark.sql(sql)
        parquet_data.write.parquet('16_jan.parquet',mode='overwrite')
        os.system('aws s3 cp 16_jan.parquet s3://i94-backup --recursive')
        break        



In [5]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1, CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider
from cassandra import ConsistencyLevel

ssl_context = SSLContext(PROTOCOL_TLSv1)
ssl_context.load_verify_locations('AmazonRootCA1.pem')
ssl_context.verify_mode = CERT_REQUIRED
auth_provider = PlainTextAuthProvider(username=str(config['APACHE_CASSANDRA_CREDS']['CASSANDRA_USERNAME']), password=str(config['APACHE_CASSANDRA_CREDS']['CASSANDRA_PASSWORD']))
cluster = Cluster(['cassandra.eu-west-1.amazonaws.com'], ssl_context=ssl_context, auth_provider=auth_provider, port=9142)

In [6]:
print('Patient...')
session = cluster.connect()
create_keyspace="""CREATE KEYSPACE IF NOT EXISTS "i94"
                   WITH REPLICATION={'class':'SingleRegionStrategy'}"""
session.execute(create_keyspace)
sleep(10)

create_table="""CREATE TABLE IF NOT EXISTS "i94".i94 (
                                                      cicid DOUBLE,
                                                      i94yr DOUBLE,
                                                      i94mon DOUBLE,
                                                      i94cit DOUBLE,
                                                      i94res DOUBLE,
                                                      i94port TEXT,
                                                      arrdate DOUBLE,
                                                      i94mode DOUBLE,
                                                      i94addr TEXT,
                                                      depdate DOUBLE,
                                                      i94bir DOUBLE,
                                                      i94visa DOUBLE,
                                                      count DOUBLE,
                                                      dtadfile DOUBLE,
                                                      visapost TEXT,
                                                      occup TEXT,
                                                      entdepa TEXT,
                                                      entdepd TEXT,
                                                      entdepu TEXT,
                                                      matflag TEXT,
                                                      biryear DOUBLE,
                                                      dtaddto TEXT,
                                                      gender TEXT,
                                                      insnum TEXT,
                                                      airline TEXT,
                                                      admnum DOUBLE,
                                                      fltno TEXT,
                                                      visatype TEXT,
                                                      id_ TEXT,
                                                      PRIMARY KEY(id_)
                ) """
session.execute(create_table)
sleep(10)
print('Table well-prepared. you can input data from dataset.')

Patient...


In [ ]:
origin_sql="""INSERT INTO "i94".i94 ("cicid","i94yr","i94mon","i94cit","i94res","i94port","arrdate","i94mode","i94addr","depdate",
                              "i94bir","i94visa","count","dtadfile","visapost","occup","entdepa","entdepd","entdepu","matflag",
                              "biryear","dtaddto","gender","insnum","airline","admnum","fltno","visatype","id_")
                              VALUES ({0},{1},{2},{3},{4},'{5}',{6},{7},'{8}',{9},
                                      {10},{11},{12},{13},'{14}','{15}','{16}','{17}','{18}','{19}',
                                      {20},'{21}','{22}','{23}','{24}',{25},'{26}','{27}','{28}')"""


total_length=len(pd.read_csv('16_jan.csv'))

with open('{}.csv'.format('16_jan'),'r') as csv:
    #Ignore the first line, which is column head.
    csv=iter(csv)
    next(csv)
    counts=0
    for each in csv:
        lists=[]
        columns=each.split(',')
        cicid=columns[0]
        i94yr=columns[1]
        i94mon=columns[2]
        i94cit=columns[3]
        i94res=columns[4]
        i94port=columns[5]
        arrdate=columns[6]
        i94mode=columns[7]
        i94addr=columns[8]
        depdate=columns[9]
        i94bir=columns[10]
        i94visa=columns[11]
        count=columns[12]
        dtadfile=columns[13]
        visapost=columns[14]
        occup=columns[15]
        entdepa=columns[16]
        entdepd=columns[17]
        entdepu=columns[18]
        matflag=columns[19]
        biryear=columns[20]
        dtaddto=columns[21]
        gender=columns[22]
        insnum=columns[23]
        airline=columns[24]
        admnum=columns[25]
        fltno=columns[26]
        visatype=columns[27]
        id_=columns[28]        
        
        original_list=[cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,
                      count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,
                      airline,admnum,fltno,visatype,id_]
        
        for each in original_list:
            try:
                each = float(each)
                lists.append(each)
            except:
                if each == '':
                    each = None
                    lists.append(each)
                else:
                    lists.append(each)
                    continue                            
        
        formated_sql=origin_sql.format(lists[0],lists[1],lists[2],lists[3],lists[4],lists[5],lists[6],lists[7],lists[8],lists[9],
                      lists[10],lists[11],lists[12],lists[13],lists[14],lists[15],lists[16],lists[17],lists[18],lists[19],
                      lists[20],lists[21],lists[22],lists[23],lists[24],lists[25],lists[26],lists[27],lists[28])
        
        sql=session.prepare(formated_sql)
        sql.consistency_level = ConsistencyLevel.LOCAL_QUORUM        
        session.execute(sql)
        
        print('Import row{} complete.'.format(counts) + '{} remaining.'.format(total_length-counts))
        counts+=1
    


Import row0 complete.2847924 remaining.
Import row1 complete.2847923 remaining.
Import row2 complete.2847922 remaining.
Import row3 complete.2847921 remaining.
Import row4 complete.2847920 remaining.
Import row5 complete.2847919 remaining.
Import row6 complete.2847918 remaining.
Import row7 complete.2847917 remaining.
Import row8 complete.2847916 remaining.
Import row9 complete.2847915 remaining.
Import row10 complete.2847914 remaining.
Import row11 complete.2847913 remaining.
Import row12 complete.2847912 remaining.
Import row13 complete.2847911 remaining.
Import row14 complete.2847910 remaining.
Import row15 complete.2847909 remaining.
Import row16 complete.2847908 remaining.
Import row17 complete.2847907 remaining.
Import row18 complete.2847906 remaining.
Import row19 complete.2847905 remaining.
Import row20 complete.2847904 remaining.
Import row21 complete.2847903 remaining.
Import row22 complete.2847902 remaining.
Import row23 complete.2847901 remaining.
Import row24 complete.2847

Import row199 complete.2847725 remaining.
Import row200 complete.2847724 remaining.
Import row201 complete.2847723 remaining.
Import row202 complete.2847722 remaining.
Import row203 complete.2847721 remaining.
Import row204 complete.2847720 remaining.
Import row205 complete.2847719 remaining.
Import row206 complete.2847718 remaining.
Import row207 complete.2847717 remaining.
Import row208 complete.2847716 remaining.
Import row209 complete.2847715 remaining.
Import row210 complete.2847714 remaining.
Import row211 complete.2847713 remaining.
Import row212 complete.2847712 remaining.
Import row213 complete.2847711 remaining.
Import row214 complete.2847710 remaining.
Import row215 complete.2847709 remaining.
Import row216 complete.2847708 remaining.
Import row217 complete.2847707 remaining.
Import row218 complete.2847706 remaining.
Import row219 complete.2847705 remaining.
Import row220 complete.2847704 remaining.
Import row221 complete.2847703 remaining.
Import row222 complete.2847702 rem

Import row398 complete.2847526 remaining.
Import row399 complete.2847525 remaining.
Import row400 complete.2847524 remaining.
Import row401 complete.2847523 remaining.
Import row402 complete.2847522 remaining.
Import row403 complete.2847521 remaining.
Import row404 complete.2847520 remaining.
Import row405 complete.2847519 remaining.
Import row406 complete.2847518 remaining.
Import row407 complete.2847517 remaining.
Import row408 complete.2847516 remaining.
Import row409 complete.2847515 remaining.
Import row410 complete.2847514 remaining.
Import row411 complete.2847513 remaining.
Import row412 complete.2847512 remaining.
Import row413 complete.2847511 remaining.
Import row414 complete.2847510 remaining.
Import row415 complete.2847509 remaining.
Import row416 complete.2847508 remaining.
Import row417 complete.2847507 remaining.
Import row418 complete.2847506 remaining.
Import row419 complete.2847505 remaining.
Import row420 complete.2847504 remaining.
Import row421 complete.2847503 rem

Import row595 complete.2847329 remaining.
Import row596 complete.2847328 remaining.
Import row597 complete.2847327 remaining.
Import row598 complete.2847326 remaining.
Import row599 complete.2847325 remaining.
Import row600 complete.2847324 remaining.
Import row601 complete.2847323 remaining.
Import row602 complete.2847322 remaining.
Import row603 complete.2847321 remaining.
Import row604 complete.2847320 remaining.
Import row605 complete.2847319 remaining.
Import row606 complete.2847318 remaining.
Import row607 complete.2847317 remaining.
Import row608 complete.2847316 remaining.
Import row609 complete.2847315 remaining.
Import row610 complete.2847314 remaining.
Import row611 complete.2847313 remaining.
Import row612 complete.2847312 remaining.
Import row613 complete.2847311 remaining.
Import row614 complete.2847310 remaining.
Import row615 complete.2847309 remaining.
Import row616 complete.2847308 remaining.
Import row617 complete.2847307 remaining.
Import row618 complete.2847306 rem

Import row791 complete.2847133 remaining.
Import row792 complete.2847132 remaining.
Import row793 complete.2847131 remaining.
Import row794 complete.2847130 remaining.
Import row795 complete.2847129 remaining.
Import row796 complete.2847128 remaining.
Import row797 complete.2847127 remaining.
Import row798 complete.2847126 remaining.
Import row799 complete.2847125 remaining.
Import row800 complete.2847124 remaining.
Import row801 complete.2847123 remaining.
Import row802 complete.2847122 remaining.
Import row803 complete.2847121 remaining.
Import row804 complete.2847120 remaining.
Import row805 complete.2847119 remaining.
Import row806 complete.2847118 remaining.
Import row807 complete.2847117 remaining.
Import row808 complete.2847116 remaining.
Import row809 complete.2847115 remaining.
Import row810 complete.2847114 remaining.
Import row811 complete.2847113 remaining.
Import row812 complete.2847112 remaining.
Import row813 complete.2847111 remaining.
Import row814 complete.2847110 rem

...

Import row70930 complete.2776994 remaining.
Import row70931 complete.2776993 remaining.
Import row70932 complete.2776992 remaining.
Import row70933 complete.2776991 remaining.
Import row70934 complete.2776990 remaining.
Import row70935 complete.2776989 remaining.
Import row70936 complete.2776988 remaining.
Import row70937 complete.2776987 remaining.
Import row70938 complete.2776986 remaining.
Import row70939 complete.2776985 remaining.
Import row70940 complete.2776984 remaining.
Import row70941 complete.2776983 remaining.
Import row70942 complete.2776982 remaining.
Import row70943 complete.2776981 remaining.
Import row70944 complete.2776980 remaining.
Import row70945 complete.2776979 remaining.
Import row70946 complete.2776978 remaining.
Import row70947 complete.2776977 remaining.
Import row70948 complete.2776976 remaining.
Import row70949 complete.2776975 remaining.
Import row70950 complete.2776974 remaining.
Import row70951 complete.2776973 remaining.
Import row70952 complete.2776972

Import row71118 complete.2776806 remaining.
Import row71119 complete.2776805 remaining.
Import row71120 complete.2776804 remaining.
Import row71121 complete.2776803 remaining.
Import row71122 complete.2776802 remaining.
Import row71123 complete.2776801 remaining.
Import row71124 complete.2776800 remaining.
Import row71125 complete.2776799 remaining.
Import row71126 complete.2776798 remaining.
Import row71127 complete.2776797 remaining.
Import row71128 complete.2776796 remaining.
Import row71129 complete.2776795 remaining.
Import row71130 complete.2776794 remaining.
Import row71131 complete.2776793 remaining.
Import row71132 complete.2776792 remaining.
Import row71133 complete.2776791 remaining.
Import row71134 complete.2776790 remaining.
Import row71135 complete.2776789 remaining.
Import row71136 complete.2776788 remaining.
Import row71137 complete.2776787 remaining.
Import row71138 complete.2776786 remaining.
Import row71139 complete.2776785 remaining.
Import row71140 complete.2776784

Import row71308 complete.2776616 remaining.
Import row71309 complete.2776615 remaining.
Import row71310 complete.2776614 remaining.
Import row71311 complete.2776613 remaining.
Import row71312 complete.2776612 remaining.
Import row71313 complete.2776611 remaining.
Import row71314 complete.2776610 remaining.
Import row71315 complete.2776609 remaining.
Import row71316 complete.2776608 remaining.
Import row71317 complete.2776607 remaining.
Import row71318 complete.2776606 remaining.
Import row71319 complete.2776605 remaining.
Import row71320 complete.2776604 remaining.
Import row71321 complete.2776603 remaining.
Import row71322 complete.2776602 remaining.
Import row71323 complete.2776601 remaining.
Import row71324 complete.2776600 remaining.
Import row71325 complete.2776599 remaining.
Import row71326 complete.2776598 remaining.
Import row71327 complete.2776597 remaining.
Import row71328 complete.2776596 remaining.
Import row71329 complete.2776595 remaining.
Import row71330 complete.2776594

Import row71497 complete.2776427 remaining.
Import row71498 complete.2776426 remaining.
Import row71499 complete.2776425 remaining.
Import row71500 complete.2776424 remaining.
Import row71501 complete.2776423 remaining.
Import row71502 complete.2776422 remaining.
Import row71503 complete.2776421 remaining.
Import row71504 complete.2776420 remaining.
Import row71505 complete.2776419 remaining.
Import row71506 complete.2776418 remaining.
Import row71507 complete.2776417 remaining.
Import row71508 complete.2776416 remaining.
Import row71509 complete.2776415 remaining.
Import row71510 complete.2776414 remaining.
Import row71511 complete.2776413 remaining.
Import row71512 complete.2776412 remaining.
Import row71513 complete.2776411 remaining.
Import row71514 complete.2776410 remaining.
Import row71515 complete.2776409 remaining.
Import row71516 complete.2776408 remaining.
Import row71517 complete.2776407 remaining.
Import row71518 complete.2776406 remaining.
Import row71519 complete.2776405

Import row71685 complete.2776239 remaining.
Import row71686 complete.2776238 remaining.
Import row71687 complete.2776237 remaining.
Import row71688 complete.2776236 remaining.
Import row71689 complete.2776235 remaining.
Import row71690 complete.2776234 remaining.
Import row71691 complete.2776233 remaining.
Import row71692 complete.2776232 remaining.
Import row71693 complete.2776231 remaining.
Import row71694 complete.2776230 remaining.
Import row71695 complete.2776229 remaining.
Import row71696 complete.2776228 remaining.
Import row71697 complete.2776227 remaining.
Import row71698 complete.2776226 remaining.
Import row71699 complete.2776225 remaining.
Import row71700 complete.2776224 remaining.
Import row71701 complete.2776223 remaining.
Import row71702 complete.2776222 remaining.
Import row71703 complete.2776221 remaining.
Import row71704 complete.2776220 remaining.
Import row71705 complete.2776219 remaining.
Import row71706 complete.2776218 remaining.
Import row71707 complete.2776217

In [ ]:
while True:
    try:
        os.remove('16_jan.csv')
    except:
        break

In [19]:
global_land_temperature_url = 's3://srk-data-eng-capstone/GlobalLandTemperaturesByCity.csv'
airport_codes_url = 's3://srk-data-eng-capstone/airport-codes_csv.csv'
us_city_demographics_url = 's3://srk-data-eng-capstone/us-cities-demographics.csv'

In [20]:
global_land_temperature = pd.read_csv(global_land_temperature_url)
airport_codes=pd.read_csv(airport_codes_url)
us_city_demographics=pd.read_csv(us_city_demographics_url, sep=';')

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
us_cities = global_land_temperature['Country']=='United States'
global_land_temperature = global_land_temperature[us_cities]
global_land_temperature = global_land_temperature.drop(['Country'],axis=1)
start_date = '2012-01-01'
end_date = '2012-12-01'
period = (global_land_temperature['dt']>=start_date & global_land_temperature['dt']<=end_date)
global_land_temperature=global_land_temperature[period]

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.